In [15]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
using CSV
using JSON
using UUIDs
using Serialization
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(dirname(pwd()) * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### Simulations of Water Regulation Procedure

We would like to set up a skeleton for the day-to-day decision making of hydropower producers and the Water Regulation Company. The main steps of the Water Regulation company consist of:

* Determining a Reference Flow for the next nomination period (Inflow Forecasting)
* Receiving pre-bid nominations and calculating adjusted flow, power swaps, etc. -> Send these parameters back to the participants
* Recalculate based on renominations by power producers after bidding

For the Power Producers / Participants:  

* Fetch and Calculate all Parameters for the Water Regulation Procedure (Individual Reservoir Level etc.)
* Price and Inflow Forecasting (Fix Price Points)
* Medium-Term Hydropower Model Calculation, obtain Water Value Cuts
* Decision Model 1: Bidding and Pre-Market Clearing Nomination
* Decision Model 2: Short-Term Optimization and Renomination
* Decision Model 3: Fixed Water Flow scheduling and real time balancing

This constitutes one round of simulation.


In [16]:
filepath_Ljungan = dirname(pwd()) * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = dirname(pwd()) *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = dirname(pwd()) * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)

println("The reservoir system consists of $([r.dischargepoint for r in R])")
println("Downstream of the reservoirs we have power plants $([k.name for k in K])")
for j in J
    println("$(j.name) owns $(j.plants), which constitutes a participation rate of: \n ", j.participationrate)
end
println(J[3])

The reservoir system consists of ["Flasjon", "Holsmjon"]
Downstream of the reservoirs we have power plants ["Flasjo", "Trangfors", "Ratan", "Turinge", "Bursnas", "Parteboda", "Hermansboda", "Ljunga", "Nederede", "Skallbole", "Matfors", "Viforsen", "Jarnvagsforsen"]


Sydkraft owns HydropowerPlant[Flasjo, Trangfors, Ratan, Turinge, Bursnas], which constitutes a participation rate of: 
 ________________________________
Holsmjon | 0.0     
Flasjon  | 1.84    

Fortum owns HydropowerPlant[Parteboda, Hermansboda, Ljunga], which constitutes a participation rate of: 
 ________________________________
Holsmjon | 0.8200000000000001
Flasjon  | 0.8200000000000001

Statkraft owns HydropowerPlant[Nederede, Skallbole, Matfors, Viforsen, Jarnvagsforsen], which constitutes a participation rate of: 
 ________________________________
Holsmjon | 1.86    
Flasjon  | 1.86    

Statkraft


#### Load Data

Other than the river system, the relevant data for optimization is organized in Dataframes.
We load in cleaned versions of the data.

In [17]:
filepath_prices = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Spot Prices\\prices_df.csv"
filepath_inflows = dirname(pwd()) * "\\Inflow Forecasting\\Data\\Inflow\\Data from Flasjoen and Holmsjoen.csv"

price_data = prepare_pricedata(filepath_prices)
inflow_data = prepare_inflowdata(filepath_inflows)

first(price_data, 10)

Row,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,Average,season,Weekday,CalendarWeek
,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,String,Int64
1,2020-12-15,17.57,17.57,16.92,16.88,18.95,20.39,22.31,24.59,31.41,31.49,31.62,30.52,35.9,38.1,40.08,43.08,37.32,51.13,47.54,40.77,35.2,28.45,23.67,21.86,30.1383,Winter,Weekday,51
2,2020-12-16,20.48,19.32,19.0,19.02,20.19,21.98,22.71,32.71,48.37,43.68,42.01,37.92,36.56,36.42,39.16,46.54,46.4,39.41,35.78,24.74,22.78,22.15,21.48,20.12,30.7887,Winter,Weekday,51
3,2020-12-17,19.03,17.93,17.53,17.28,17.97,20.0,22.09,23.64,26.79,25.28,24.43,24.3,23.67,23.99,23.97,25.06,25.38,26.07,24.22,23.16,22.19,21.97,20.76,18.99,22.3208,Winter,Weekday,51
4,2020-12-18,16.96,16.65,16.17,16.3,17.3,19.39,19.99,22.43,23.74,23.96,22.82,22.5,22.44,22.23,22.42,22.91,23.11,23.83,22.59,21.01,19.43,19.04,17.82,15.92,20.4567,Winter,Weekday,51
5,2020-12-19,16.48,15.98,15.47,15.31,15.3,15.52,15.87,15.92,16.48,17.01,17.36,17.47,17.27,17.29,17.33,17.54,17.96,18.39,17.77,17.03,15.92,15.69,15.22,13.65,16.4679,Winter,Weekend,51
6,2020-12-20,13.0,12.04,11.79,11.65,11.79,11.98,12.53,12.7,13.25,14.28,15.37,15.94,15.83,15.37,15.98,16.84,17.09,17.29,17.06,16.3,15.32,15.23,14.65,13.02,14.4292,Winter,Weekend,51
7,2020-12-21,11.9,11.2,11.5,11.73,12.81,14.37,16.67,19.74,20.6,19.92,19.32,19.39,19.44,19.2,19.45,20.65,20.55,20.35,19.49,18.03,16.17,14.47,13.18,8.66,16.6163,Winter,Weekday,52
8,2020-12-22,7.15,5.51,4.77,4.44,4.84,7.73,12.82,15.97,17.87,18.0,18.09,18.01,18.01,17.57,18.08,18.62,19.67,19.87,19.79,18.01,16.88,16.05,15.05,13.71,14.4379,Winter,Weekday,52
9,2020-12-23,13.38,13.1,12.9,12.93,13.14,14.72,15.99,18.59,19.21,19.51,18.97,19.33,19.55,19.4,19.66,19.96,20.56,21.79,20.77,18.95,17.82,16.94,16.16,15.07,17.4333,Winter,Weekday,52


### Price Forecasting

Price is sensitive to various factors. Daytime, Weekday and time of year play a relevant role in how prices usually behave.

* For the bidding and short-term optimization problems, part of the uncertainty lies in unknown hourly prices.

* For the medium term problem, we are interested in daily (average) prices.

* We want to obtain Price Points which will be important in the analysis from the historic data.

* We need penalty parameters for using the balancing markets, which could be obtained from using maximum / minimum values fromm observed spot market prices / generated scenarios.

In [18]:
const scenario_count_prices_short = 10
const scenario_count_prices_medium = 2
const stage_count_short = 8
const stage_count_medium = 52
const price_point_count = 5
const T = 24

PriceScenariosMedium = Price_Scenarios_Medium(price_data, scenario_count_prices_medium)
PriceScenariosShort = Price_Scenarios_Short(price_data, scenario_count_prices_short, stage_count_short)
PPoints = Create_Price_Points(price_data, 5)
mu_up, mu_down = BalanceParameters(PriceScenariosShort)

println(PPoints)

[0.0, 31.612541666666665, 53.16795833333333, 76.75770833333334, 111.81674999999998, 180.42366666666663, 440.0]


### Inflow Forecasting

Inflow has extreme seasonal differences, depending on geographical location.  
For the Ljungan River System we could observe that most inflow occurs during the spring melt around May.

In [19]:
const ColumnReservoir = Dict{Reservoir, String}(R[1] => "Flasjon Inflow", R[2] => "Holmsjon Inflow")
const scenario_count_inflows_weekly = 2
const scenario_count_inflows_short = 1


InflowScenariosShort = Inflow_Scenarios_Short(inflow_data, 1, ColumnReservoir, R, stage_count_short, scenario_count_prices_short)
InflowScenariosMedium = Inflow_Scenarios_Medium(inflow_data, ColumnReservoir, scenario_count_inflows_weekly, R)
first(InflowScenariosShort, 10)

8-element Vector{Pair{Int64, Dict{Reservoir, Vector{Float64}}}}:
 5 => Dict(Holsmjon => [5.8, 10.7, 23.5, 0.0, 0.0, 13.9, 0.0, 51.6, 5.8, 25.0], Flasjon => [5.25, 8.41, 8.87, 6.82, 10.84, 5.33, 5.33, 10.43, 11.29, 6.26])
 4 => Dict(Holsmjon => [20.5, 33.2, 2.2, 0.0, 24.4, 25.0, 24.5, 2.7, 0.0, 38.9], Flasjon => [7.98, 8.2, 11.32, 10.2, 10.25, 7.85, 6.91, 6.2, 6.75, 7.2])
 6 => Dict(Holsmjon => [5.8, 20.4, 19.1, 0.0, 33.1, 4.4, 28.2, 21.5, 6.7, 6.8], Flasjon => [7.9, 11.29, 7.01, 6.82, 5.79, 4.06, 8.87, 8.29, 10.61, 6.91])
 7 => Dict(Holsmjon => [10.7, 22.6, 20.5, 52.4, 0.0, 12.2, 33.6, 0.0, 55.3, 46.4], Flasjon => [7.01, 5.33, 10.87, 8.8, 9.7, 5.25, 1.88, 9.92, 6.75, 6.63])
 2 => Dict(Holsmjon => [10.7, 10.0, 23.0, 25.4, 0.0, 23.5, 77.5, 4.1, 0.0, 6.7], Flasjon => [11.6, 13.49, 7.62, 9.42, 10.87, 9.1, 8.29, 10.61, 7.9, 6.67])
 8 => Dict(Holsmjon => [6.8, 4.0, 9.3, 28.2, 23.7, 16.6, 4.0, 0.0, 43.9, 3.0], Flasjon => [8.23, 10.6, 11.0, 5.8, 7.98, 5.79, 14.73, 10.29, 8.87, 7.35])
 3 => Dic

### Create Uncertainty Set

Depending on the Problem, we create the uncertainty sets from both price and inflow Scenarios.
This becomes more difficult for the Anticipatory Bidding Problem.

In [20]:
Ω_medium, P_medium =  create_Ω_medium(PriceScenariosMedium, InflowScenariosMedium, R);
Ω_NA, P_NA = create_Ω_Nonanticipatory(PriceScenariosShort, InflowScenariosShort, R, stage_count_short)
Ω_NA[1]

"""
    create_Ω_Anticipatory(PriceScenariosShort, InflowScenariosShort, j, R)

    create Uncertainty Set for the Anticipatory Problem: 
    Addiitionally to the Nonanticipatory Case, we associate deterministic sets of inflows and prices with other producers' nominations.
    The Uncertainty set is extended entrywise by the others' nomination. 
"""

function create_Ω_Anticipatory(PriceScenariosShort, InflowScenariosShort, OthersNominations, j::Participant, R::Vector{Reservoir}, stage_count = stage_count_bidding)
    Ω = Dict(i => [(inflow = Dict(r => InflowScenariosShort[i][r][j] for r in R), price = c) for c in PriceScenariosShort[i] for j in eachindex(InflowScenariosShort[i][R[1]])] for i in 1:stage_count)
    P = Dict(s => [1/length(eachindex(Ω[s])) for i in eachindex(Ω[s])] for s in 1:stage_count)
    return Ω, P
end
# Ω_A, P_A = create_Ω_Anticipatory()

create_Ω_Anticipatory (generic function with 2 methods)

### Reference Flow

To calculate a reference flow for each reservoir, the Water Regulation Company has to combine information from inflow, historical reservoir trajectories and current reservoir level.  

In [21]:
l_traj, f = AverageReservoirLevel(R, inflow_data)
Qref = CalculateReferenceFlow(R, l_traj, f, 2)

Dict{Reservoir, Float64} with 2 entries:
  Holsmjon => -0.0456916
  Flasjon  => -0.734399

## Participants

Every Participant individually plans their bidding and production. Irrespective of the strategy, only limited information is available. Internal to the participants problems are 

* Price Forecasts
* Nomination to Water Regulation Company
* Bidding at Electricity Market

### Medium Term Problem

The Medium Term / Seasonal Optimization Problem is necessary to obtain Cuts for the Water Value Function.

Therefor the point of a functioin containing this decision model is to return cut coefficients of the Water Value Function.

* What are the inputs to the Medium Term problem?
* How are the cuts calculated?
* How many cuts should be calculated?

In [22]:
const iterations = 10

const savepath_watervalue = "C:\\Users\\lenna\\OneDrive - NTNU\\Code Master Thesis\\Inflow Forecasting\\WaterValue"

WaterValueDictionary_j, WaterValueDictionary_O = MediumModelsAllParticipants(J, R, Ω_medium, P_medium, stage_count_medium, iterations)
println(WaterValueDictionary_j)
println(WaterValueDictionary_O)
SaveMediumModel(savepath_watervalue * "\\Participant.jls", WaterValueDictionary_j)
SaveMediumModel(savepath_watervalue * "\\OtherParticipant.jls", WaterValueDictionary_O)

  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.


  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.


  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.


  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.


Dict{Participant, SDDP.PolicyGraph}(Statkraft => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
, Fortum => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
, Sydkraft => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
)
Dict{Participant, SDDP.PolicyGraph}(Statkraft => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
, Fortum => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
, Sydkraft => A policy graph with 52 nodes.
 Node indices: 1, ..., 52
)


In [23]:
NameToParticipant = Dict{String, Participant}(j.name => j for j in J)
MediumModelDictionary_j_loaded = ReadMediumModel(savepath_watervalue * "\\Participant.jls", NameToParticipant);
MediumModelDictionary_O_loaded = ReadMediumModel(savepath_watervalue * "\\OtherParticipant.jls", NameToParticipant);


cuts = Dict(j => ReservoirLevelCuts(R, j.plants, j, f, 2, stage_count_short) for j in J)
WaterCuts = Dict(j => WaterValueCuts(MediumModelDictionary_j_loaded[j], cuts[j], 2) for j in J)

Reservoir[Holsmjon, Flasjon]
Dict{Reservoir, Float64}[________________________________
Flasjon  | 1823.471746031746
, ________________________________
Flasjon  | 1754.0  
, ________________________________
Flasjon  | 2743.471746031746
, ________________________________
Flasjon  | 2674.0  
]Reservoir[Holsmjon, Flasjon]
Dict{Reservoir, Float64}[________________________________
Holsmjon | 1365.0  
Flasjon  | 1514.0  
, ________________________________
Holsmjon | 350.0857142857143
Flasjon  | 1583.471746031746
, ________________________________
Holsmjon | 1510.0857142857142
Flasjon  | 1514.0  
, ________________________________
Holsmjon | 350.0857142857143
Flasjon  | 2743.471746031746
, ________________________________
Holsmjon | 350.0857142857143
Flasjon  | 1514.0  
, ________________________________
Holsmjon | 1365.0  
Flasjon  | 2674.0  
, ________________________________
Holsmjon | 350.0857142857143
Flasjon  | 2674.0  
, ________________________________
Holsmjon | 205.0   
Flasjon  | 15

Dict{Participant, Dict{Dict{Reservoir, Float64}, NamedTuple{(:e1, :e2), Tuple{Float64, Dict{Symbol, Float64}}}}} with 3 entries:
  Statkraft => Dict(________________________________…
  Fortum    => Dict(________________________________…
  Sydkraft  => Dict(________________________________…

### Bidding Problem

* Every Power Producer solves their own Bidding problem with individual Water Value Cuts to obtain their bidding curves and nomination.
* The solutions are taken as input for the calculation of the adjusted Flow and Power Swap
* The Market clears, and the obligations for the next production day are revealed.

In [40]:
HourlyBiddingCurves = Dict{Participant, Dict{Int64, Vector{Float64}}}()
Qnoms = Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}()

for j in J
    println(j)
    Qnom, HourlyBiddingCurve = Nonanticipatory_Bidding(R, j, PPoints, Ω_NA, P_NA, Qref, cuts[j], WaterCuts[j], iterations, mu_up, mu_down, T, stage_count_short)
    HourlyBiddingCurves[j] = HourlyBiddingCurve
    for r in R
        if j.participationrate[r] > 0
            Qnoms[(participant = j, reservoir = r)] = Qnom[r]
        else
            Qnoms[(participant = j, reservoir = r)] = Qref[r]
        end
    end
end


Sydkraft
-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 8
  state variables : 176
  scenarios       : 1.00000e+14
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [356, 813]
  AffExpr in MOI.EqualTo{Float64}                                               : [7, 61]
  AffExpr in MOI.GreaterThan{Float64}                                           : [115, 115]
  AffExpr in MOI.LessThan{Float64}                                              : [148, 388]
  VariableRef in MOI.GreaterThan{Float64}                                       : [171, 389]
  VariableRef in MOI.LessThan{Float64}                                       

numerical stability report
  matrix range     [3e-04, 2e+02]
  objective range  [1e+00, 4e+02]
  bounds range     [7e+00, 8e+06]
  rhs range        [7e-01, 4e+05]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1   1.127494e+06  2.051678e+06  7.800000e-01       709   1


         3   1.052425e+06  1.433488e+06  2.380000e+00      2127   1


         5   1.482456e+06  1.333142e+06  4.156000e+00      3545   1

...


         3   2.116432e+06  3.383851e+06  2.679000e+00      2127   1


         5   3.622050e+06  3.254778e+06  4.469000e+00      3545   1


         7   2.802634e+06  3.238981e+06  6.301000e+00      4963   1


         9   2.988802e+06  3.199293e+06  8.073000e+00      6381   1


        10   2.662136e+06  3.176910e+06  8.948000e+00      7090   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 8.948000e+00
total solves   : 7090
best bound     :  3.176910e+06
simulation ci  :  2.703206e+06 ± 4.728178e+05
numeric issues : 0
-------------------------------------------------------------------



### Water Regulation

Without Updating the Reservoir Levels, the adjusted flow and Power Swaps are calculated.

In [41]:
Qadj, _, P_Swap, _, _, _ = water_regulation(Qnoms, Qref, T, false)
println(Qadj)
println(P_Swap)

________________________________
Holsmjon | 99.52823775683166
Flasjon  | 21.034829374003113

Dict{Participant, Dict{Reservoir, Float64}}(Statkraft => ________________________________
Holsmjon | -4.161300198092764
Flasjon  | -39.12478263564579
, Fortum => ________________________________
Holsmjon | 4.16130019809271
Flasjon  | -17.248560086682556
, Sydkraft => ________________________________
Holsmjon | -0.0    
Flasjon  | 56.37334272232835
)


### Market Clearing

A market clearing price is generated. (From the historical prices)
For each hourly bid, the obligation is determined.

* Sample a Price for that day (Save Price)
* Perform Linear Interpolation between Price Points in which the hourly price lands, and determine the obligation from that
* Give Hourly Obligations to each user (Dictionary of Participants and hourly obligations)

In [42]:
price = Price_Scenarios_Short(price_data, 1, stage_count_short)[1][1]
Obligations = MarketClearing(price, HourlyBiddingCurves, PPoints, J, T)
for j in J
    println(Obligations[j])
end

[0.0, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.12999999999997, 184.13, 15.785571356774186, 184.13000000000002, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13000000000002, 184.13, 184.13]
[0.0, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.12999999999997, 184.13, 15.785571356774186, 184.13000000000002, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13000000000002, 184.13, 184.13]
[0.0, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.12999999999997, 184.13, 15.785571356774186, 184.13000000000002, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13, 184.13000000000002, 184.13, 184.13]


### Short Term Optimization

Feed Power Swap and Nomination into Short Term Optimization Model

In [43]:
"""
    OthersNomination(Qnoms, Qadj)

    From own nomination and others nomination, obtain what the other participants have nominated aggregated.
    Create a Dictionary with that information for each participant.

"""
function OthersNomination(Qnoms::Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}, Qadj::Dict{Reservoir, Float64}, J::Vector{Participant}, R::Vector{Reservoir})
    QnomO = Dict{Participant, Dict{Reservoir, Float64}}(j => Dict( r => 0.0 for r in R) for j in J)
    for j in J
        O, K_O = OtherParticipant(J, j, R)
        for r in R
            QnomO[j][r] = (Qadj[r] * (j.participationrate[r] + O.participationrate[r]) - Qnoms[(participant = j, reservoir = r)] * j.participationrate[r] )/O.participationrate[r]
        end
    end
    return QnomO
end

QnomO = OthersNomination(Qnoms, Qadj, J, R)
Qnoms_2 = Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}()
for j in J
    println(j)
    Qnom =ShortTermScheduling(R, j, J, Qref, Obligations[j], price, QnomO[j], Ω_NA, P_NA, cuts[j], WaterCuts[j], iterations, mu_up,mu_down, T, stage_count_short) 
    for r in R
        if j.participationrate[r] > 0
            Qnoms_2[(participant = j, reservoir = r)] = Qnom[r]
        else
            Qnoms_2[(participant = j, reservoir = r)] = Qref[r]
        end
    end
end

Sydkraft
Reservoir[Flasjon]
SydkraftReservoir[Flasjon]
Plants: HydropowerPlant[Flasjo, Trangfors, Ratan, Turinge, Bursnas]
HydropowerPlant[Parteboda, Hermansboda, Ljunga, Nederede, Skallbole, Matfors, Viforsen, Jarnvagsforsen]
-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 8
  state variables : 7
  scenarios       : 1.00000e+14
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [404, 463]
  AffExpr in MOI.EqualTo{Float64}                                               : [13, 39]
  AffExpr in MOI.GreaterThan{Float64}                                           : [115, 123]
  AffExpr in MOI.LessThan{Float64}            


numerical stability report
  matrix range     [7e-02, 2e+02]
  objective range  [1e+00, 4e+02]
  bounds range     [7e+00, 2e+08]
  rhs range        [7e-01, 4e+05]
  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1   2.852847e+06  2.389763e+06  3.840001e-01       709   1


         4   2.005133e+06  2.389763e+06  1.639000e+00      2836   1


         7   2.131550e+06  2.389763e+06  2.770000e+00      4963   1


...

numerical stability report
  matrix range     [7e-02, 2e+02]
  objective range  [1e+00, 4e+02]
  bounds range     [1e+01, 3e+08]
  rhs range        [5e-02, 1e+06]
  - bounds range contains large coefficients
Very large or small absolute values of coefficients
can cause numerical stability issues. Consider
reformulating the model.
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


         1   3.414900e+06  4.031435e+06  3.829999e-01       709   1


         4   3.539018e+06  4.031435e+06  1.625000e+00      2836   1


         7   5.679008e+06  4.031435e+06  2.894000e+00      4963   1


        10   3.904665e+06  4.031435e+06  4.176000e+00      7090   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 4.176000e+00
total solves   : 7090
best bound     :  4.031435e+06
simulation ci  :  4.112364e+06 ± 5.883351e+05
numeric issues : 0
-------------------------------------------------------------------

Nomination: 
________________________________
Holsmjon | 14.51213434555875
Flasjon  | 204.04439313414292



### Second Round of Water Regulation

This time, the reservoirs are updated accordingly

In [45]:
Qnoms_2

Qadj_2, _, P_Swap_2, _, _, _ = water_regulation(Qnoms_2, Qref, T, false)

(________________________________
Holsmjon | 10.071854433857947
Flasjon  | 148.2484540784729
, ________________________________
Holsmjon | 158.32030851233083
Flasjon  | 148.2484540784729
, Dict{Participant, Dict{Reservoir, Float64}}(Statkraft => ________________________________
Holsmjon | -4.513732344347785
Flasjon  | 38.17683961413431
, Fortum => ________________________________
Holsmjon | 4.513732344347785
Flasjon  | -19.67103325092723
, Sydkraft => ________________________________
Holsmjon | -0.0    
Flasjon  | -18.50580636320707
), Viforsen | 0.0     
Ratan    | 24.34036578159379
Ljunga   | 18.774358436976776
Skallbole | 0.0     
Flasjo   | 13.719115258716498
Matfors  | 0.0     
Nederede | 0.0     
Bursnas  | 3.0978647358392095
Hermansboda | 4.172079652661506
Parteboda | 11.264615062186067
Jarnvagsforsen | 0.0     
Trangfors | 31.863751568631866
Turinge  | 8.408489997277854
Viforsen | 0.0     
Ratan    | 4.121573830693027
Ljunga   | 0.0     
Skallbole | 0.0     
Flasjo   | 2.323068

Statkraft
______________________________
Fortum
______________________________
Sydkraft
______________________________


### Real Time Balancing and Scheduling

In [56]:
j = J[1]
_, z_up, z_down = RealTimeBalancing(
    R,
    j,
    Qadj_2,
    P_Swap_2[j],
    T,
    mu_up,
    mu_down,
    Obligations[j])

println(z_down, z_up)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 149.83862228001874, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0][18.50580636320707, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 0.0, 18.505806363207117, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.50580636320706, 18.505806363207117, 18.50580636320706, 18.50580636320706]


# Simulation

For a preset amount of rounds, we simulate the reservoir system through the functions and structures as described above. It is important to have a way to save results adequately! This encapsulates the calculations, and evaluations and visualizations can take place at a later point in time.